<a href="https://colab.research.google.com/github/simasaadi/noaa-nyc-climate-2020-2025/blob/main/notebooks/02_station_profiles_and_trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02 – Station Climate Profiles and Trends (NOAA NYC 2020–2025)


## 1. Setup and data loading


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

url_clean = "https://raw.githubusercontent.com/simasaadi/noaa-nyc-climate-2020-2025/main/data/processed/noaa_nyc_annual_clean.csv"

df = pd.read_csv(url_clean)

df.head()


,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,CDSD,CLDD,DSND,...,TSUN,WDF2,WDF5,WSF2,WSF5,annual_range,total_degree_days,wetness_index,heat_extreme_days,cold_extreme_days
0,US1NJHD0018,"KEARNY 1.7 NNW, NJ US",40.774342,-74.137109,25.6,2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.16,NaN,NaN
1,US1NJHD0018,"KEARNY 1.7 NNW, NJ US",40.774342,-74.137109,25.6,2024,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.54,NaN,NaN
2,US1NJES0018,"MAPLEWOOD TWP 0.9 SE, NJ US",40.724466,-74.259542,72.5,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.05,NaN,NaN
3,US1NJES0018,"MAPLEWOOD TWP 0.9 SE, NJ US",40.724466,-74.259542,72.5,2024,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.04,NaN,NaN
4,USW00094728,"NY CITY CENTRAL PARK, NY US",40.778980,-73.969250,42.7,2020,NaN,1306.0,1306.0,9.0,...,NaN,NaN,NaN,NaN,NaN,82.0,8666.0,58.19,20.0,6.0


## 2. Station climate profiles (aggregated 2020–2025)


In [2]:
profile_cols = [
    "TAVG", "TMAX", "TMIN",
    "PRCP", "SNOW",
    "annual_range",
    "total_degree_days",
    "wetness_index",
    "heat_extreme_days",
    "cold_extreme_days"
]

station_profiles = (
    df
    .groupby(["STATION", "NAME", "LATITUDE", "LONGITUDE", "ELEVATION"])[profile_cols]
    .mean()
    .reset_index()
)

station_profiles.head()


,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,TAVG,TMAX,TMIN,PRCP,SNOW,annual_range,total_degree_days,wetness_index,heat_extreme_days,cold_extreme_days
0,US1NJBG0015,"NORTH ARLINGTON 0.7 WNW, NJ US",40.791492,-74.139790,17.7,NaN,NaN,NaN,55.4175,NaN,NaN,NaN,55.4175,NaN,NaN
1,US1NJBG0017,"GLEN ROCK 0.7 SSE, NJ US",40.951090,-74.118264,28.0,NaN,NaN,NaN,52.5650,NaN,NaN,NaN,52.5650,NaN,NaN
2,US1NJBG0018,"PALISADES PARK 0.2 WNW, NJ US",40.848094,-74.000247,21.3,NaN,NaN,NaN,48.0300,NaN,NaN,NaN,48.0300,NaN,NaN
3,US1NJBG0023,"OAKLAND 0.9 SSE, NJ US",41.019050,-74.233383,149.4,NaN,NaN,NaN,48.5850,28.4,NaN,NaN,62.7850,NaN,NaN
4,US1NJBG0030,"OAKLAND 1.0 ESE, NJ US",41.025324,-74.223632,109.4,NaN,NaN,NaN,55.1150,NaN,NaN,NaN,55.1150,NaN,NaN


In [18]:
# Focus on the 15 warmest stations and show a clean, analytical view
top15_tavg = (
    station_profiles
    .sort_values("TAVG", ascending=False)   # warmest first
    .head(15)                               # take top 15
    .sort_values("TAVG")                    # sort again for nicer bar order
)

fig = px.bar(
    top15_tavg,
    x="TAVG",
    y="NAME",
    orientation="h",
    color="TAVG",
    color_continuous_scale="Viridis",
    title= "Top 15 Warmest Stations – NYC Metro Area (Annual Avg Temp, 2020–2025)",

    hover_data=["LATITUDE", "LONGITUDE", "ELEVATION"]
)

fig.update_layout(
    xaxis_title="Average Annual Temperature (°F)",
    yaxis_title="Station",
    height=600,
    margin=dict(l=200, r=40, t=80, b=40)  # extra left margin for long names
)
fig.update_traces(texttemplate="%{x:.1f}°F", textposition="outside")
mean_tavg = station_profiles["TAVG"].mean()
fig.add_vline(
    x=mean_tavg,
    line_dash="dot",
    line_color="black",
    annotation_text=f"Metro Avg: {mean_tavg:.1f}°F",
    annotation_position="top right"
)



fig.show()



In [19]:
# --- Top 15 wettest stations: advanced visualization ---

# 1. Filter to the 15 stations with highest average precipitation
top15_prcp = (
    station_profiles
    .sort_values("PRCP", ascending=False)   # wettest first
    .head(15)                               # take top 15
    .sort_values("PRCP")                    # re-sort for nice bar order
)

# 2. Compute metro-wide average precipitation for reference line
metro_prcp_mean = station_profiles["PRCP"].mean()

# 3. Horizontal bar chart with color encoding + rich hover
fig = px.bar(
    top15_prcp,
    x="PRCP",
    y="NAME",
    orientation="h",
    color="PRCP",
    color_continuous_scale="Blues",
    title="Top 15 Wettest Stations (Average Annual Precipitation, 2020–2025)",
    hover_data=["LATITUDE", "LONGITUDE", "ELEVATION"]
)

# 4. Add labels, axis formatting, and layout tweaks
fig.update_traces(
    texttemplate="%{x:.1f} in",
    textposition="outside"
)

fig.update_layout(
    xaxis_title="Average Annual Precipitation (inches)",
    yaxis_title="Station",
    height=600,
    margin=dict(l=220, r=40, t=80, b=40),  # extra left margin for long names
    coloraxis_colorbar_title="PRCP (in)"
)

# 5. Add vertical line at metro-wide mean precipitation
fig.add_vline(
    x=metro_prcp_mean,
    line_dash="dot",
    line_color="black",
    annotation_text=f"Metro Avg: {metro_prcp_mean:.1f} in",
    annotation_position="top right"
)

fig.show()


## 3. Station-level trends (temperature and heat extremes)


In [22]:
# Recompute trends with proper NaN handling

def compute_trend(group: pd.DataFrame, col: str) -> float:
    """
    Compute a simple linear trend (slope per year) for column `col`
    within one station, ignoring NaNs.
    """
    sub = group[["DATE", col]].dropna()   # remove NaNs first

    # Need at least 2 distinct years with data to fit a trend
    if sub["DATE"].nunique() < 2:
        return np.nan

    x = sub["DATE"].values
    y = sub[col].values
    slope, intercept = np.polyfit(x, y, 1)
    return slope

trend_df = (
    df
    .groupby(["STATION", "NAME"], as_index=False)
    .apply(lambda g: pd.Series({
        "tavg_trend_per_year": compute_trend(g, "TAVG"),
        "dx90_trend_per_year": compute_trend(g, "DX90"),
    }))
    .reset_index(drop=True)
)

trend_df.head(), trend_df["tavg_trend_per_year"].describe()



/tmp/ipython-input-3564744457.py:22: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



(       STATION                            NAME  tavg_trend_per_year  \
 0  US1NJBG0015  NORTH ARLINGTON 0.7 WNW, NJ US                  NaN   
 1  US1NJBG0017        GLEN ROCK 0.7 SSE, NJ US                  NaN   
 2  US1NJBG0018   PALISADES PARK 0.2 WNW, NJ US                  NaN   
 3  US1NJBG0023          OAKLAND 0.9 SSE, NJ US                  NaN   
 4  US1NJBG0030          OAKLAND 1.0 ESE, NJ US                  NaN   
 
    dx90_trend_per_year  
 0                  NaN  
 1                  NaN  
 2                  NaN  
 3                  NaN  
 4                  NaN  ,
 count    14.000000
 mean      0.095714
 std       0.155252
 min      -0.130000
 25%      -0.045000
 50%       0.115000
 75%       0.207500
 max       0.350000
 Name: tavg_trend_per_year, dtype: float64)

In [23]:
station_profiles_trend = station_profiles.merge(
    trend_df,
    on=["STATION", "NAME"],
    how="left"
)

station_profiles_trend[["NAME", "tavg_trend_per_year"]].head()


,NAME,tavg_trend_per_year
0,"NORTH ARLINGTON 0.7 WNW, NJ US",NaN
1,"GLEN ROCK 0.7 SSE, NJ US",NaN
2,"PALISADES PARK 0.2 WNW, NJ US",NaN
3,"OAKLAND 0.9 SSE, NJ US",NaN
4,"OAKLAND 1.0 ESE, NJ US",NaN


In [6]:
station_profiles_trend = station_profiles.merge(
    trend_df,
    on=["STATION", "NAME"],
    how="left"
)

station_profiles_trend.head()


,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,TAVG,TMAX,TMIN,PRCP,SNOW,annual_range,total_degree_days,wetness_index,heat_extreme_days,cold_extreme_days,tavg_trend_per_year,dx90_trend_per_year
0,US1NJBG0015,"NORTH ARLINGTON 0.7 WNW, NJ US",40.791492,-74.139790,17.7,NaN,NaN,NaN,55.4175,NaN,NaN,NaN,55.4175,NaN,NaN,NaN,NaN
1,US1NJBG0017,"GLEN ROCK 0.7 SSE, NJ US",40.951090,-74.118264,28.0,NaN,NaN,NaN,52.5650,NaN,NaN,NaN,52.5650,NaN,NaN,NaN,NaN
2,US1NJBG0018,"PALISADES PARK 0.2 WNW, NJ US",40.848094,-74.000247,21.3,NaN,NaN,NaN,48.0300,NaN,NaN,NaN,48.0300,NaN,NaN,NaN,NaN
3,US1NJBG0023,"OAKLAND 0.9 SSE, NJ US",41.019050,-74.233383,149.4,NaN,NaN,NaN,48.5850,28.4,NaN,NaN,62.7850,NaN,NaN,NaN,NaN
4,US1NJBG0030,"OAKLAND 1.0 ESE, NJ US",41.025324,-74.223632,109.4,NaN,NaN,NaN,55.1150,NaN,NaN,NaN,55.1150,NaN,NaN,NaN,NaN


In [20]:
# --- Stations with strongest warming trend in average temperature ---

# 1. Keep only stations where we actually estimated a slope
trend_non_null = station_profiles_trend.dropna(subset=["tavg_trend_per_year"]).copy()

# 2. Take the 10 stations with the largest positive warming trend
top10_warming = (
    trend_non_null
    .sort_values("tavg_trend_per_year", ascending=False)
    .head(10)
    .sort_values("tavg_trend_per_year")   # sort again for nicer bar order
)

top10_warming[["NAME", "tavg_trend_per_year"]]


,NAME,tavg_trend_per_year


In [24]:
# --- Stations with strongest warming trend in average temperature ---

trend_non_null = station_profiles_trend.dropna(subset=["tavg_trend_per_year"]).copy()

top10_warming = (
    trend_non_null
    .sort_values("tavg_trend_per_year", ascending=False)
    .head(10)
    .sort_values("tavg_trend_per_year")
)

fig = px.bar(
    top10_warming,
    x="tavg_trend_per_year",
    y="NAME",
    orientation="h",
    color="tavg_trend_per_year",
    color_continuous_scale="RdBu_r",
    title="Stations With Strongest Warming Trend in Average Temperature (°F/year)",
    hover_data=["LATITUDE", "LONGITUDE", "ELEVATION"]
)

fig.update_traces(
    texttemplate="%{x:.2f} °F/yr",
    textposition="outside"
)

fig.update_layout(
    xaxis_title="Trend in Average Temperature (°F per year)",
    yaxis_title="Station",
    height=600,
    margin=dict(l=220, r=40, t=80, b=40),
    coloraxis_colorbar_title="°F/yr"
)

fig.add_vline(
    x=0,
    line_dash="dash",
    line_color="black",
    annotation_text="No trend",
    annotation_position="top left"
)

fig.show()



In [25]:
# --- Stations with strongest increase in hot days (DX90) ---

# 1. Keep only stations with a valid DX90 trend
dx90_non_null = station_profiles_trend.dropna(subset=["dx90_trend_per_year"]).copy()

# Quick sanity check (optional, but useful)
dx90_non_null[["NAME", "dx90_trend_per_year"]] \
    .sort_values("dx90_trend_per_year", ascending=False) \
    .head(15)


,NAME,dx90_trend_per_year
55,"TETERBORO AIRPORT, NJ US",1.0
56,"WESTCHESTER CO AIRPORT, NY US",0.4
45,"HARRISON, NJ US",-0.1
43,"BOONTON 1 SE, NJ US",-0.2
54,"NY CITY CENTRAL PARK, NY US",-0.3
57,"JFK INTERNATIONAL AIRPORT, NY US",-0.6
51,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",-0.8
44,"CANOE BROOK, NJ US",-1.2
49,"SYOSSET, NY US",-1.5
52,"CALDWELL ESSEX CO AIRPORT, NJ US",-2.5


In [26]:
import plotly.express as px

# 2. Take the 10 stations with the strongest increase in hot days
top10_dx90 = (
    dx90_non_null
    .sort_values("dx90_trend_per_year", ascending=False)
    .head(10)
    .sort_values("dx90_trend_per_year")   # re-sort for clean bar order
)

fig = px.bar(
    top10_dx90,
    x="dx90_trend_per_year",
    y="NAME",
    orientation="h",
    color="dx90_trend_per_year",
    color_continuous_scale="OrRd",
    title="Stations With Strongest Increase in Days ≥ 90°F (days/year)",
    hover_data=["LATITUDE", "LONGITUDE", "ELEVATION"]
)

# 3. Label bars and format layout
fig.update_traces(
    texttemplate="%{x:.2f} days/yr",
    textposition="outside"
)

fig.update_layout(
    xaxis_title="Trend in Days ≥ 90°F (days per year)",
    yaxis_title="Station",
    height=600,
    margin=dict(l=220, r=40, t=80, b=40),
    coloraxis_colorbar_title="days/yr"
)

# 4. Reference line at zero (no change in hot days)
fig.add_vline(
    x=0,
    line_dash="dash",
    line_color="black",
    annotation_text="No trend",
    annotation_position="top left"
)

fig.show()



## 3. Station-level trends (temperature and heat extremes)


In [9]:
def compute_trend(group, col):
    # group is df for a single station
    years = group["DATE"].values
    values = group[col].values
    if len(np.unique(years)) < 2:
        return np.nan
    slope, intercept = np.polyfit(years, values, 1)
    return slope

trend_df = (
    df
    .groupby(["STATION", "NAME"])
    .apply(lambda g: pd.Series({
        "tavg_trend_per_year": compute_trend(g, "TAVG"),
        "dx90_trend_per_year": compute_trend(g, "DX90")
    }))
    .reset_index()
)

trend_df.head()


/tmp/ipython-input-583598943.py:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,STATION,NAME,tavg_trend_per_year,dx90_trend_per_year
0,US1NJBG0015,"NORTH ARLINGTON 0.7 WNW, NJ US",NaN,NaN
1,US1NJBG0017,"GLEN ROCK 0.7 SSE, NJ US",NaN,NaN
2,US1NJBG0018,"PALISADES PARK 0.2 WNW, NJ US",NaN,NaN
3,US1NJBG0023,"OAKLAND 0.9 SSE, NJ US",NaN,NaN
4,US1NJBG0030,"OAKLAND 1.0 ESE, NJ US",NaN,NaN


In [10]:
station_profiles_trend = station_profiles.merge(
    trend_df,
    on=["STATION", "NAME"],
    how="left"
)

station_profiles_trend.head()


,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,TAVG,TMAX,TMIN,PRCP,SNOW,annual_range,total_degree_days,wetness_index,heat_extreme_days,cold_extreme_days,tavg_trend_per_year,dx90_trend_per_year
0,US1NJBG0015,"NORTH ARLINGTON 0.7 WNW, NJ US",40.791492,-74.139790,17.7,NaN,NaN,NaN,55.4175,NaN,NaN,NaN,55.4175,NaN,NaN,NaN,NaN
1,US1NJBG0017,"GLEN ROCK 0.7 SSE, NJ US",40.951090,-74.118264,28.0,NaN,NaN,NaN,52.5650,NaN,NaN,NaN,52.5650,NaN,NaN,NaN,NaN
2,US1NJBG0018,"PALISADES PARK 0.2 WNW, NJ US",40.848094,-74.000247,21.3,NaN,NaN,NaN,48.0300,NaN,NaN,NaN,48.0300,NaN,NaN,NaN,NaN
3,US1NJBG0023,"OAKLAND 0.9 SSE, NJ US",41.019050,-74.233383,149.4,NaN,NaN,NaN,48.5850,28.4,NaN,NaN,62.7850,NaN,NaN,NaN,NaN
4,US1NJBG0030,"OAKLAND 1.0 ESE, NJ US",41.025324,-74.223632,109.4,NaN,NaN,NaN,55.1150,NaN,NaN,NaN,55.1150,NaN,NaN,NaN,NaN


In [28]:
fig = px.bar(
    station_profiles_trend.sort_values("tavg_trend_per_year"),
    x="NAME", y="tavg_trend_per_year",
    title="Trend in Average Temperature per Year (°F/year)",
)


In [30]:
trend_non_null = station_profiles_trend.dropna(subset=["tavg_trend_per_year"]).copy()

fig = px.histogram(
    trend_non_null,
    x="tavg_trend_per_year",
    nbins=12,
    marginal="box",
    color_discrete_sequence=["#4C72B0"],
    title="Distribution of Temperature Trends Across NOAA Stations (2020–2025)"
)

fig.update_layout(
    xaxis_title="Temperature Trend (°F per year)",
    yaxis_title="Number of Stations",
    height=450,
    bargap=0.05,
    template="plotly_white"
)

fig.add_vline(
    x=0,
    line_dash="dash",
    line_color="black",
    annotation_text="Zero Trend",
    annotation_position="top right"
)

fig.show()


In [31]:
# Keep only stations with a valid DX90 (hot days) trend
dx90_non_null = station_profiles_trend.dropna(subset=["dx90_trend_per_year"]).copy()

# Optional: inspect the strongest increases
dx90_non_null[["NAME", "dx90_trend_per_year"]] \
    .sort_values("dx90_trend_per_year", ascending=False) \
    .head(15)


,NAME,dx90_trend_per_year
55,"TETERBORO AIRPORT, NJ US",1.0
56,"WESTCHESTER CO AIRPORT, NY US",0.4
45,"HARRISON, NJ US",-0.1
43,"BOONTON 1 SE, NJ US",-0.2
54,"NY CITY CENTRAL PARK, NY US",-0.3
57,"JFK INTERNATIONAL AIRPORT, NY US",-0.6
51,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",-0.8
44,"CANOE BROOK, NJ US",-1.2
49,"SYOSSET, NY US",-1.5
52,"CALDWELL ESSEX CO AIRPORT, NJ US",-2.5


In [32]:
import plotly.express as px

# Top 10 stations with the largest increase in very hot days
top10_dx90 = (
    dx90_non_null
    .sort_values("dx90_trend_per_year", ascending=False)
    .head(10)
    .sort_values("dx90_trend_per_year")   # sort again for clean bar order
)

fig = px.bar(
    top10_dx90,
    x="dx90_trend_per_year",
    y="NAME",
    orientation="h",
    color="dx90_trend_per_year",
    color_continuous_scale="OrRd",
    title="Stations With Strongest Increase in Days ≥ 90°F (days/year)",
    hover_data=["LATITUDE", "LONGITUDE", "ELEVATION"]
)

# Label bars
fig.update_traces(
    texttemplate="%{x:.2f} days/yr",
    textposition="outside"
)

# Layout polish
fig.update_layout(
    xaxis_title="Trend in Days ≥ 90°F (days per year)",
    yaxis_title="Station",
    height=600,
    margin=dict(l=220, r=40, t=80, b=40),
    coloraxis_colorbar_title="days/yr"
)

# Reference line at zero (no change in hot days)
fig.add_vline(
    x=0,
    line_dash="dash",
    line_color="black",
    annotation_text="No trend",
    annotation_position="top left"
)

fig.show()



## 4. Time series for selected stations


In [13]:
# See what station names we have
df["NAME"].unique()


array(['KEARNY 1.7 NNW, NJ US', 'MAPLEWOOD TWP 0.9 SE, NJ US',
       'NY CITY CENTRAL PARK, NY US', 'EATONTOWN 1.2 NE, NJ US',
       'NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US',
       'OAKLAND 0.9 SSE, NJ US', 'HICKSVILLE 1.3 ENE, NY US',
       'LAGUARDIA AIRPORT, NY US', 'WOOD RIDGE 0.2 N, NJ US',
       'SYOSSET, NY US', 'CALDWELL ESSEX CO AIRPORT, NJ US',
       'FARMINGDALE REPUBLIC AIRPORT, NY US',
       'CEDAR GROVE TWP 0.4 W, NJ US', 'HARRISON, NJ US',
       'HAWTHORNE 1.0 SSE, NJ US', 'WOODBRIDGE TWP 1.1 NNE, NJ US',
       'WOODBRIDGE TWP 1.1 ESE, NJ US', 'LITTLE FALLS TWP 0.5 WNW, NJ US',
       'CRANFORD TWP 1.1 NNW, NJ US', 'WAYNE TWP 4.2 NNW, NJ US',
       'MASSAPEQUA 1.1 SE, NY US', 'CENTERPORT, NY US',
       'RIVER EDGE 0.4 NNE, NJ US', 'CANOE BROOK, NJ US',
       'NEW PROVIDENCE 0.8 ESE, NJ US', 'OAKLAND 1.0 ESE, NJ US',
       'LITTLE SILVER 0.3 NNW, NJ US', 'HIGHLAND PARK 0.5 E, NJ US',
       'BELLMORE 0.9 SSE, NY US', 'KEARNY 1.7 NW, NJ US',
       'HOWAR

In [34]:
import numpy as np
import plotly.graph_objects as go

# subset already defined earlier:
# mask = (...)
# subset = df[mask].copy()
# subset["DATE"] = subset["DATE"].astype(int)
# subset = subset.sort_values(["NAME", "DATE"])

fig = go.Figure()

for name in subset["NAME"].unique():
    sub = subset[subset["NAME"] == name].copy()

    # 1) Observed annual TAVG (lines + markers)
    fig.add_trace(
        go.Scatter(
            x=sub["DATE"],
            y=sub["TAVG"],
            mode="lines+markers",
            name=f"{name} (observed)",
        )
    )

    # 2) Linear trend line for each station (dashed)
    if sub["DATE"].nunique() >= 2:
        z = np.polyfit(sub["DATE"], sub["TAVG"], 1)   # slope, intercept
        trend_y = np.polyval(z, sub["DATE"])

        fig.add_trace(
            go.Scatter(
                x=sub["DATE"],
                y=trend_y,
                mode="lines",
                name=f"{name} trend",
                line=dict(dash="dash", width=2),
                showlegend=True,
            )
        )

fig.update_layout(
    title="Average Annual Temperature and Trends for Key NYC Stations (2020–2025)",
    xaxis_title="Year",
    yaxis_title="Average Annual Temperature (°F)",
    legend_title="Series",
    height=520,
)

fig.show()
